# Customer Lifetime  Value (CLV) Prediction

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lifetimes

from datetime import datetime
from dateutil.relativedelta import relativedelta

import os
from google.cloud import bigquery

### Import Data from BigQuery

In [26]:
# Import order data
gcr_project_id = os.getenv('GCR_CLV_PROJECT_ID')
QUERY  = f"""
WITH order_values AS (
    SELECT 
      order_id,
      SUM(sale_price) as order_value
    FROM `{gcr_project_id}.thelook_ecommerce.order_items`
    GROUP BY order_id
    ORDER BY order_id
)
SELECT 
  orders.order_id,
  orders.user_id,
  users.first_name,
  users.last_name,
  users.email,
  orders.created_at,
  orders.status,
  order_values.order_value
FROM `{gcr_project_id}.thelook_ecommerce.orders` AS orders
    LEFT JOIN `{gcr_project_id}.thelook_ecommerce.users` AS users ON orders.user_id = users.id
    LEFT JOIN order_values on orders.order_id = order_values.order_id
ORDER BY orders.order_id;
"""

client = bigquery.Client()

df = client.query_and_wait(QUERY).to_dataframe()

/Users/axel/ecommerce-project/env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


### Import data from local files

In [45]:
df_orders = pd.read_csv('data/orders.csv')
df_order_items = pd.read_csv('data/order_items.csv')
df_users = pd.read_csv('data/users.csv')

order_values = df_order_items.groupby('order_id', as_index=False).agg({'sale_price':'sum'}).sort_values(by='order_id').rename(columns={'sale_price':'order_value'})
df = pd.merge(df_orders[['order_id','user_id','created_at','status']],
              df_users[['id','first_name','last_name','email']], how='left', left_on='user_id', right_on='id')
df = pd.merge(df,
              order_values[['order_id','order_value']], how='left', on='order_id').sort_values(by='order_id').reset_index(drop=True)
df['created_at'] = df.created_at.apply(lambda x : datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
df

,order_id,user_id,created_at,status,id,first_name,last_name,email,order_value
0,1,3,2021-11-24,Cancelled,3,Mackenzie,Dixon,mackenziedixon@example.org,19.98
1,2,3,2023-09-10,Processing,3,Mackenzie,Dixon,mackenziedixon@example.org,68.00
2,3,4,2024-02-06,Shipped,4,Amanda,Cooper,amandacooper@example.org,9.12
3,4,4,2023-01-05,Cancelled,4,Amanda,Cooper,amandacooper@example.org,71.95
4,5,4,2020-11-09,Complete,4,Amanda,Cooper,amandacooper@example.org,15.99
...,...,...,...,...,...,...,...,...,...
124944,124945,99999,2024-10-30,Returned,99999,Ian,Carrillo,iancarrillo@example.com,16.90
124945,124946,99999,2024-06-25,Processing,99999,Ian,Carrillo,iancarrillo@example.com,39.00
124946,124947,100000,2024-11-24,Shipped,100000,Cindy,Ferguson,cindyferguson@example.com,231.50
124947,124948,100000,2024-09-23,Processing,100000,Cindy,Ferguson,cindyferguson@example.com,38.00


### Formatting Data - Calculate Frequency, Recency, Customer Age, and Customer Monetary Value.

In [46]:
# Reformat 'created_at' column the exclude time of day
from datetime import datetime
df2 = df.copy()
df2['created_at'] = df2.created_at.apply(lambda x : x.date())
df2.head()

,order_id,user_id,created_at,status,id,first_name,last_name,email,order_value
0,1,3,2021-11-24,Cancelled,3,Mackenzie,Dixon,mackenziedixon@example.org,19.98
1,2,3,2023-09-10,Processing,3,Mackenzie,Dixon,mackenziedixon@example.org,68.00
2,3,4,2024-02-06,Shipped,4,Amanda,Cooper,amandacooper@example.org,9.12
3,4,4,2023-01-05,Cancelled,4,Amanda,Cooper,amandacooper@example.org,71.95
4,5,4,2020-11-09,Complete,4,Amanda,Cooper,amandacooper@example.org,15.99


In [47]:
# Get Customer Summary Data : Frequency, Recency, Monetary Value
df_rfm  = lifetimes.utils.summary_data_from_transaction_data(df2, 'user_id', 'created_at',
                                                            freq='D', include_first_transaction = False)
df_rfm['monetary_value'] = df2.groupby('user_id')[['order_value']].mean()
df_rfm.head()

,frequency,recency,T,monetary_value
user_id,,,,
3,1.0,655.0,1138.0,43.990
4,3.0,1424.0,1518.0,49.015
5,0.0,0.0,889.0,15.100
7,0.0,0.0,10.0,158.270
8,1.0,674.0,1384.0,41.455


### Defining Utility Functions for Evaluating CLV Prediction Model Performance

This section contains the definition of the following functions:

* `get_train_test_rfm` - get training and testing datasets for evaluating a predictive model. The training dataset contains RFM data for shoppers whose first purchase was made during the training period. The testing dataset contains the true RFM and equity data of these shoppers during the testing period.
* `get_pred_equity` - predict the future equity of users whose data was used to fit the model during the prediction period.
* `eval_predictions` - calculate metrics to evaluate the models predictions.

In [48]:
# Dataset creation functions
def get_train_test_rfm(df : pd.DataFrame, 
                        train_period_start : datetime.date,
                        train_period_end : datetime.date,
                        prediction_period_duration : int=12):
    """Get RFM summary dataframes for the training & testing periods.
    
    Args:
        df - dataframe containing order data
        train_period_start - start of training period
        train_period_end - end of training period
        prediction_period_duration - duration of the prediction period in months
        
    Returns:
        Tuple[df_train_rfm, df_test_rfm, df_all_rfm]
    """
            
    # Create copy of dataset & reformat 'created_at' column
    df1 = df.copy()
    df1['created_at'] = df1.created_at.apply(lambda x : x.date())

    # Calculate end of prediction period and assert ensure that it ends before the maximum order date
    prediction_period_end = train_period_end + relativedelta(months=prediction_period_duration)
    assert df1.created_at.max() > prediction_period_end, f"Prediction period ends in the future - we do not have the data to evaluate these predictions : try reducing the prediction period duration or choosing an earlier training period end date"

    # Filter out users who were 'alive' before training period started
    first_order_dates = df1.groupby('user_id')['created_at'].min()
    valid_users = first_order_dates[(first_order_dates >= train_period_start) & (first_order_dates <= train_period_end)].index
    df1 = df1[df1['user_id'].isin(valid_users)]

    # Select data from training & testing periods
    df_train = df1[(df1.created_at <= train_period_end)]
    df_test = df1[(df1.created_at > train_period_end) & (df1.created_at <= prediction_period_end)]

    # Get RFM summary data from training period
    df_train_rfm  = lifetimes.utils.summary_data_from_transaction_data(df_train, 'user_id', 'created_at',
                                                                           freq='D', include_first_transaction = False)
    df_train_rfm = pd.merge(df_train_rfm, df_train.groupby('user_id')['order_value'].agg(['mean', 'sum']), 
                            how='left', on='user_id').rename(columns={'mean' : 'monetary_value', 'sum' : 'revenue'})
    
    # Get RFM summary data from testing period
    df_test_rfm = lifetimes.utils.summary_data_from_transaction_data(df_test, 'user_id', 'created_at',
                                                                     freq='D', include_first_transaction = True)
    df_test_rfm = pd.merge(df_test_rfm, df_test.groupby('user_id')['order_value'].agg('sum'), 
                           how='left', on='user_id').rename(columns={'frequency' : 'true_purchases', 'order_value':'true_equity'})
    
    # Combine training & testing RFM data
    df_all_rfm = pd.merge(df_train_rfm.rename(columns={'frequency':'train_frequency', 'recency':'train_recency', 'T':'train_T'}), 
                          df_test_rfm.rename(columns={'recency':'test_recency', 'T':'test_T'}), how='left', left_index=True, right_index=True)

    
    return df_train_rfm, df_test_rfm, df_all_rfm

In [49]:
# Prediction functions
def get_pred_equity(model,
                    prediction_period_duration : int=12,
                    discount_rate : float=0.1, freq : str="D"):
    """Predict the equity of training dataset shoppers during the prediction period.
    
    Args:
        model - prediction model that has already been fitted with the training dataset
        prediction_period_duration - duration of the prediction period in months
        
    Returns:
        pred_equity - dataframe containing predicted equity for each shoppper
    """
    
    pred_equity = model.predict_clv(prediction_period_duration, discount_rate, freq).rename(columns={'clv':'pred_equity'})

    return pred_equity

In [89]:
# Evaluation functions
def eval_predictions(pred_equity : pd.Series,
                     true_equity : pd.Series):
    """Calculates metrics to evaluate equity predictions"""

    error_description = (true_equity.fillna(0)-pred_equity).describe().to_frame().T[['count', 'mean', '50%', 'min', 'max']].rename({'50%':'median'}, axis=1)
    prop_underpredicted = sum(true_equity.fillna(0) > pred_equity)/error_description['count']
    prop_overpredicted = sum(pred_equity > true_equity.fillna(0))/error_description['count']

    df_metrics = pd.concat([error_description,
                          pd.DataFrame({'prop_underpredicted' : prop_underpredicted, 'prop_overpredicted' : prop_overpredicted})], axis=1)
    return df_metrics

### Gamma-Gamma CLV Prediction Model

In [50]:
# Create class for Gamma-Gamma prediction model
class PredictorGGF:
    def __init__(self, df_summary):
        self.df_summary = df_summary
        self.correlation = self.df_summary[self.df_summary.frequency != 0][['monetary_value', 'frequency']].corr().values[0,1]

        return print(f"Correlation between shopper frequency & monetary value is : {float(self.correlation):.5f}.")

    def fit_bgf(self, penalty_coef : float=0.01):

        self.bgf = lifetimes.BetaGeoFitter(penalty_coef)
        self.bgf.fit(self.df_summary['frequency'],
                    self.df_summary['recency'],
                    self.df_summary['T'])

        print(f"Beta-Gamma model successfully fitted")
        return self.bgf.summary

    def fit_ggf(self, penalty_coef : float=0.01):
        assert self.correlation < 0.1, f"Correlation between frequency and monetary value for returning customers is {self.correlation} - this is quite high and may cause poor predictions"

        self.ggf = lifetimes.GammaGammaFitter(penalty_coef)
        self.ggf.fit(self.df_summary[self.df_summary.frequency != 0]['frequency'],
                     self.df_summary[self.df_summary.frequency != 0]['monetary_value'])

        print(f"Gamma-Gamma model successfully fitted")
        if float(self.ggf.params_['q']) < 1:
            print("Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.\nFix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.")

        return self.ggf.summary
    
    def predict_clv(self, time : int=12, discount_rate : float=0.1, freq : str="D"):
        """Predict Customer Lifetime Value
        Args:
            time (float, optional) – the lifetime expected for the user in months. Default: 12
            discount_rate (float, optional) – the monthly adjusted discount rate. Default: 0.01
            freq (string, optional) – {“D”, “H”, “M”, “W”} for day, hour, month, week. This represents what unit of time your T is measure in.

        Returns:
            Series – Series object with customer ids as index and the estimated customer lifetime values as values
        """

        # Predict customer lifetime value
        clv_preds_df = self.ggf.customer_lifetime_value(
                            self.bgf,
                            self.df_summary['frequency'],
                            self.df_summary['recency'],
                            self.df_summary['T'],
                            self.df_summary['monetary_value'],
                            time=time,
                            discount_rate=discount_rate,
                            freq=freq
                        ).to_frame()
        
        return clv_preds_df

In [51]:
ggf_model = PredictorGGF(df_rfm)
penalty_val = 0.01
bgf_summary = ggf_model.fit_bgf(penalty_coef=penalty_val)
ggf_summary = ggf_model.fit_ggf(penalty_coef=penalty_val)
clv_preds = ggf_model.predict_clv()
clv_preds

Correlation between shopper frequency & monetary value is : -0.00295.
Beta-Gamma model successfully fitted
Gamma-Gamma model successfully fitted
Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.
Fix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.


,clv
user_id,
3,6.123401
4,18.007963
5,-3.755855
7,-10.634346
8,4.582409
...,...
99996,59.147754
99997,32.130195
99998,12.919142


In [53]:
# Experiment 1 : Training Period = 1/1/2022 - 30/4/2024, Testing Period = 1/5/2024 - 31/12/2024
df_train, df_test, df_all = get_train_test_rfm(df, train_period_start=datetime(2022,1,1).date(), train_period_end=datetime(2024,4,30).date(), prediction_period_duration=8)

# Fit GGF model to training data
ggf_model_exp1 = PredictorGGF(df_train)
penalty_val = 0.01
bgf_summary = ggf_model_exp1.fit_bgf(penalty_coef=penalty_val)
ggf_summary = ggf_model_exp1.fit_ggf(penalty_coef=penalty_val)

# Predict shopper equity during prediction period
pred_equity = get_pred_equity(ggf_model_exp1, prediction_period_duration=8)

# Add predicted equity to testing period data
df_all = pd.merge(df_all, pred_equity, how='left', left_on='user_id', right_index=True)
df_all

Correlation between shopper frequency & monetary value is : -0.01515.
Beta-Gamma model successfully fitted
Gamma-Gamma model successfully fitted
Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.
Fix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.


,train_frequency,train_recency,train_T,monetary_value,revenue,true_purchases,test_recency,test_T,true_equity,pred_equity
user_id,,,,,,,,,,
5,0.0,0.0,639.0,15.100,15.10,NaN,NaN,NaN,NaN,-3.138140
11,0.0,0.0,510.0,15.950,15.95,3.0,81.0,86.0,249.82,-3.527605
13,0.0,0.0,354.0,194.490,194.49,NaN,NaN,NaN,NaN,-4.151363
15,0.0,0.0,530.0,10.990,10.99,NaN,NaN,NaN,NaN,-3.460986
21,0.0,0.0,383.0,19.990,19.99,NaN,NaN,NaN,NaN,-4.019166
...,...,...,...,...,...,...,...,...,...,...
99985,1.0,108.0,332.0,37.245,74.49,NaN,NaN,NaN,NaN,7.127179
99993,0.0,0.0,49.0,173.830,173.83,NaN,NaN,NaN,NaN,-6.356546
99995,0.0,0.0,264.0,210.020,210.02,NaN,NaN,NaN,NaN,-4.623701


In [90]:
# Assess overall predictions
eval_predictions(df_all['pred_equity'], df_all['true_equity'])

# Assess cases where predicted equity is > 0 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 0]['pred_equity'], df_all[df_all.pred_equity > 0]['true_equity'])

# Asses cases where prediced equity is > 50 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 50]['pred_equity'], df_all[df_all.pred_equity > 50]['true_equity'])

# Asses cases where shoppers were less than 90 days old at end of training period & predicted equity is greater than 0 - Model is still overpredicting in these cases, but to a less extent
eval_predictions(df_all[(df_all.pred_equity > 0) & (df_all.train_T < 90)]['pred_equity'], df_all[(df_all.pred_equity > 0) & (df_all.train_T < 90)]['true_equity'])

,count,mean,median,min,max,prop_underpredicted,prop_overpredicted
0,5864.0,22.158875,-5.211336,-151.644545,985.075092,0.282401,0.717599


### Raw Monetary Value CLV Prediction Model

In [85]:
# Create class for Raw Monetary Value prediction model
import lifetimes.utils
class PredictorRawMonetary:
    def __init__(self, df_summary):
        self.df_summary = df_summary
        self.correlation = self.df_summary[self.df_summary.frequency != 0][['monetary_value', 'frequency']].corr().values[0,1]

        return print(f"Correlation between shopper frequency & monetary value is : {float(self.correlation):.5f}.")

    def fit_bgf(self, penalty_coef : float=0.01):

        self.bgf = lifetimes.BetaGeoFitter(penalty_coef)
        self.bgf.fit(self.df_summary['frequency'],
                    self.df_summary['recency'],
                    self.df_summary['T'])

        print(f"Beta-Gamma model successfully fitted")
        return self.bgf.summary
    
    def predict_clv(self, time : int=12, discount_rate : float=0.1, freq : str="D"):
        """Predict Customer Lifetime Value
        Args:
            time (float, optional) – the lifetime expected for the user in months. Default: 12
            discount_rate (float, optional) – the monthly adjusted discount rate. Default: 0.01
            freq (string, optional) – {“D”, “H”, “M”, “W”} for day, hour, month, week. This represents what unit of time your T is measure in.

        Returns:
            Series – Series object with customer ids as index and the estimated customer lifetime values as values
        """

        # Predict customer lifetime value
        clv_preds_df = lifetimes.utils._customer_lifetime_value(
                            self.bgf,
                            self.df_summary['frequency'],
                            self.df_summary['recency'],
                            self.df_summary['T'],
                            self.df_summary['monetary_value'],
                            time=time,
                            discount_rate=discount_rate,
                            freq=freq
                        ).to_frame()
        
        return clv_preds_df

In [86]:
rmv_model = PredictorRawMonetary(df_rfm)
penalty_val = 0.01
bgf_summary = rmv_model.fit_bgf(penalty_coef=penalty_val)
clv_preds = rmv_model.predict_clv()
clv_preds

Correlation between shopper frequency & monetary value is : -0.00295.
Beta-Gamma model successfully fitted


,clv
user_id,
3,5.108702
4,17.006118
5,1.300879
7,38.606514
8,3.803879
...,...
99996,52.158126
99997,27.645938
99998,10.863410


In [87]:
# Experiment 1 : Training Period = 1/1/2022 - 30/4/2024, Testing Period = 1/5/2024 - 31/12/2024
df_train, df_test, df_all = get_train_test_rfm(df, train_period_start=datetime(2022,1,1).date(), train_period_end=datetime(2024,4,30).date(), prediction_period_duration=8)

# Fit GGF model to training data
rmv_model_exp1 = PredictorRawMonetary(df_train)
penalty_val = 0.01
bgf_summary = rmv_model_exp1.fit_bgf(penalty_coef=penalty_val)

# Predict shopper equity during prediction period
pred_equity = get_pred_equity(rmv_model_exp1, prediction_period_duration=8)

# Add predicted equity to testing period data
df_all = pd.merge(df_all, pred_equity, how='left', left_on='user_id', right_index=True)
df_all

Correlation between shopper frequency & monetary value is : -0.01515.
Beta-Gamma model successfully fitted


,train_frequency,train_recency,train_T,monetary_value,revenue,true_purchases,test_recency,test_T,true_equity,pred_equity
user_id,,,,,,,,,,
5,0.0,0.0,639.0,15.100,15.10,NaN,NaN,NaN,NaN,1.092807
11,0.0,0.0,510.0,15.950,15.95,3.0,81.0,86.0,249.82,1.297583
13,0.0,0.0,354.0,194.490,194.49,NaN,NaN,NaN,NaN,18.620115
15,0.0,0.0,530.0,10.990,10.99,NaN,NaN,NaN,NaN,0.877186
21,0.0,0.0,383.0,19.990,19.99,NaN,NaN,NaN,NaN,1.852862
...,...,...,...,...,...,...,...,...,...,...
99985,1.0,108.0,332.0,37.245,74.49,NaN,NaN,NaN,NaN,5.861838
99993,0.0,0.0,49.0,173.830,173.83,NaN,NaN,NaN,NaN,25.482397
99995,0.0,0.0,264.0,210.020,210.02,NaN,NaN,NaN,NaN,22.394675


In [88]:
# Assess overall predictions
eval_predictions(df_all['pred_equity'], df_all['true_equity'])

# Asses cases where prediced equity is > 50 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 50]['pred_equity'], df_all[df_all.pred_equity > 50]['true_equity'])

# Asses cases where prediced equity is > 50 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 100]['pred_equity'], df_all[df_all.pred_equity > 100]['true_equity'])

# Asses cases where shoppers were less than 90 days old at end of training period - Model is still overpredicting in these cases, but to a less extent
eval_predictions(df_all[(df_all.train_T < 90)]['pred_equity'], df_all[(df_all.train_T < 90)]['true_equity'])

,count,mean,std,min,25%,50%,75%,max,prop_underpredicted,prop_overpredicted
0,5864.0,22.158875,87.037519,-151.644545,-13.017802,-5.211336,14.6133,985.075092,0.282401,0.717599
